<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Задача-1" data-toc-modified-id="Задача-1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Задача 1</a></span></li><li><span><a href="#Задача-2" data-toc-modified-id="Задача-2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Задача 2</a></span></li><li><span><a href="#Задача-3" data-toc-modified-id="Задача-3-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Задача 3</a></span></li></ul></div>

Homework4
Школа DataScientist, декабрь 2022, Андреева Ольга

In [7]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing

# Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad средствами библиотеки Pandas сформируйте поле выводов по каждому анализирумомому параметру. Например, по показателю pH получен результат 8.4 единицы pH при нормативе от 6 до 9 единиц pH. Т.о. по данному показателю результат анализа в норме. Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, 6 >= pH >= 9. В итоговом DataFrame столбец "Показатель" сделайте индексным.

Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).

In [40]:
headers={
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
}
def get_content(url):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url).content
    return r

url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = get_content(url)
df=pd.read_csv(io.StringIO(s.decode('UTF8')), sep=',')


### Если не работает загрузка on-line
#df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
df=df.rename(columns={'Единица измерений':'Единица_измерений', 'Результат анализа':'Результат_анализа'})

df.describe()

,Показатель,Единица_измерений,Результат_анализа,Норматив
count,14,14,14,14
unique,14,5,14,13
top,Железо (включая хлорное железо) по Fe,мг/дм3,8.4,"не более 0,3"
freq,1,10,1,2


In [41]:
#функция для поиска всех чисел (неотрицательных) в строке
def find_numbers(p_str):
    try:
        v_str=p_str.replace(',','.')
        res=re.findall('\d+(?:\.\d+)?', v_str)
        res=[float(n) for n in res]
        return res
    except ValueError:
        return None
    except AttributeError:
        return None

In [42]:
# проверка соответствия значения p_value нормативу в нормализованном виде (p_limits, p_relation)
# возможные значения p_relation:
        # 'менее': 0
        # 'не более': 1
        # 'не менее': 2
        # 'более': 3
        # 'в пределах': 4
def check_normative(p_value, p_limits, p_relation):
    try:
        if p_relation == 0:
                return (p_value<p_limits[0])
        elif p_relation == 1:
                return (p_value<=p_limits[0])
        elif p_relation == 2:
                return (p_value<=p_limits[0])
        elif p_relation == 3:
                return (p_value>p_limits[0])    
        elif p_relation == 4:
                return ((p_value>=p_limits[0])&(p_value<=p_limits[1]))
        else:
            return (None)
    except TypeError:
            return (None)

In [43]:
# проверка соответствия значения p_value нормативу в текстовом виде (p_nrmtv)
def parse_normative(p_value, p_nrmtv):
    c_templates=dict({'(\s+)?менее\s':0
                     ,'(\s+)?не(\s+)?более\s':1
                     ,'(\s+)?не(\s+)?менее\s':2
                     , '(\s+)?более':3
                     , '(\s+)?в(\s+)?пределах\s':4
                     })
    try:
        for v_tmplt in c_templates:
            if re.match(v_tmplt, p_nrmtv):
                v_limits = find_numbers(p_nrmtv)
                return(check_normative(float(p_value), v_limits, c_templates.get(v_tmplt)))
        return ('ParseNormativeError')
    except TypeError:
            return ('TypeError')
    except ValueError:
            return ('ValueError')
#print(parse_normative('7', 'в пределах 6-9'))

In [44]:
# сформирую поле выводов "соответствует нормативу" по каждому анализирумомому параметру.
df['Соответсвует нормативу']=df.apply(lambda x: parse_normative(x.Результат_анализа, x.Норматив), axis=1)

# В итоговом DataFrame столбец "Показатель" сделаю индексным.
df.set_index('Показатель', inplace=True)
df
#Замечание. Некорректно сформулирован норматив 'не более 2-3'. Не пноятно, это не более 2, не более 3, или не более 2-3=-1, или между 2 и 3?
#Текущий код интерпретирует это как не более 2.

,Единица_измерений,Результат_анализа,Норматив,Соответсвует нормативу
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,True
Запах,баллы,1,не более 2-3,True
Цветность,градусы,б/цвета,не более 30,ValueError
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,True
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",True
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",True
Нитраты (по NO3),мг/дм3,24,не более 45,True
Фосфаты (P),мг/дм3,0.36,"не более 3,5",True
Хлориды (Cl),мг/дм3,200,не более 350,True


# Задача 2



В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?

In [22]:
#зададим число экспериментов
p_exp_count=200000
#зададим параметры наполнения ящика
p_params=dict({'apple':4, 'orange':5})
p_result_required=np.array(['orange','orange','orange'])

#сгенерируем содержимое ящика
v_frut_box=[]
for i in p_params:
    v_frut_box.extend([i]*p_params.get(i))
#print(v_frut_box)

#проводим p_exp_count экспериментов, запоминаем количество положительных результатов 
v_count=0    
for i in range(p_exp_count):
    v_result=np.random.choice(v_frut_box, size=3, replace=False)
    if (np.array_equal(v_result, p_result_required)):
        v_count += 1
        
#выведем долю успешных исходов
print ('доля успешных исходов: ',v_count/p_exp_count)

доля успешных исходов:  0.11806


Эксперементальный результат совпадает с расчетным с точностью до 2 знаков

# Задача 3



Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?

В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [23]:
#зададим число экспериментов
p_exp_cnt=200000
#зададим параметры наполнения ящика
p_params=dict({'standart':7, 'not_standart':3})
p_result_required='not_standart'
p_success = 2

#сгенерируем содержимое ящика
v_parts=[]
for i in p_params:
    v_parts.extend([i]*p_params.get(i))

v_success_cnt = 0

#проводим p_exp_count экспериментов, запоминаем количество положительных результатов 
for i in range(p_exp_cnt):
    v_try_cnt = 0
    v_parts_i=v_parts.copy()
    while True:
        v_try_cnt += 1
        v_result=np.random.choice(v_parts_i)
        if v_result == p_result_required:
            break
        else:
            v_parts_i.remove(v_result) #т.к. не возвращаем деталь в ящик, перед следующим выбором ее нужно удалить из списка
    if v_try_cnt == p_success:
        v_success_cnt += 1
        
#выведем долю успешных исходов
print ('доля успешных исходов: ', v_success_cnt/p_exp_cnt)

доля успешных исходов:  0.233475


Эксперементальный результат совпадает с расчетным с точностью до 3 знаков